<a href="https://colab.research.google.com/github/sankardevisharath/amex-default-prediction/blob/master/notebooks/SGDClassifier_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%mkdir data
%cd data
%mkdir raw
%cd raw

/content/data
/content/data/raw


In [2]:
!cp /content/drive/MyDrive/amex-default-prediction/data/raw/data_2017-03.parquet /content/data/raw/

In [3]:
!unzip /content/drive/MyDrive/amex-default-prediction/data/raw/amex-default-prediction.zip train_labels.csv

Archive:  /content/drive/MyDrive/amex-default-prediction/data/raw/amex-default-prediction.zip
  inflating: train_labels.csv        


In [4]:
import numpy as np
import pandas as pd

In [5]:
TRAIN_DATA_PATH = '/content/data/raw/data_2017-03.parquet'
TRAIN_LABELS_PATH = '/content/data/raw/train_labels.csv'

In [6]:
df = pd.read_parquet(TRAIN_DATA_PATH)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395630 entries, 0 to 395629
Columns: 190 entries, customer_ID to D_145
dtypes: datetime64[ns](1), float64(185), int64(1), object(3)
memory usage: 573.5+ MB


In [8]:
df['customer_ID'].nunique()

395630

In [9]:
def read_cols(cols):
  df = pd.read_csv(TRAIN_DATA_PATH, usecols=cols)
  df = append_label(df)
  return df

def append_label(source_df):
  return pd.merge(left=source_df, right=train_labels, how='inner')

In [10]:
train_labels = pd.read_csv(TRAIN_LABELS_PATH)

In [11]:
df = append_label(df)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Columns: 191 entries, customer_ID to target
dtypes: datetime64[ns](1), float64(185), int64(2), object(3)
memory usage: 579.5+ MB


In [13]:
cat_cols = ['D_63', 'D_64','B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_66', 'D_68']

In [14]:
df[cat_cols].isnull().sum() 

D_63          0
D_64      24578
B_30        199
B_38        199
D_114     14105
D_116     14105
D_117     14105
D_120     14105
D_126     14105
D_66     352773
D_68      24578
dtype: int64

In [15]:
df[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   D_63    395630 non-null  object 
 1   D_64    371052 non-null  object 
 2   B_30    395431 non-null  float64
 3   B_38    395431 non-null  float64
 4   D_114   381525 non-null  float64
 5   D_116   381525 non-null  float64
 6   D_117   381525 non-null  float64
 7   D_120   381525 non-null  float64
 8   D_126   381525 non-null  float64
 9   D_66    42857 non-null   float64
 10  D_68    371052 non-null  float64
dtypes: float64(9), object(2)
memory usage: 36.2+ MB


In [16]:
from sklearn.preprocessing import OrdinalEncoder

In [17]:
 enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

In [18]:
df[['D_63', 'D_64']]=enc.fit_transform(df[['D_63', 'D_64']])

In [19]:
enc.categories_

[array(['CL', 'CO', 'CR', 'XL', 'XM', 'XZ'], dtype=object),
 array(['-1', 'O', 'R', 'U', None], dtype=object)]

In [20]:
df[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   D_63    395630 non-null  float64
 1   D_64    395630 non-null  float64
 2   B_30    395431 non-null  float64
 3   B_38    395431 non-null  float64
 4   D_114   381525 non-null  float64
 5   D_116   381525 non-null  float64
 6   D_117   381525 non-null  float64
 7   D_120   381525 non-null  float64
 8   D_126   381525 non-null  float64
 9   D_66    42857 non-null   float64
 10  D_68    371052 non-null  float64
dtypes: float64(11)
memory usage: 36.2 MB


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Columns: 191 entries, customer_ID to target
dtypes: datetime64[ns](1), float64(187), int64(2), object(1)
memory usage: 579.5+ MB


In [22]:
df.fillna(0, inplace=True)

In [23]:
df[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   D_63    395630 non-null  float64
 1   D_64    395630 non-null  float64
 2   B_30    395630 non-null  float64
 3   B_38    395630 non-null  float64
 4   D_114   395630 non-null  float64
 5   D_116   395630 non-null  float64
 6   D_117   395630 non-null  float64
 7   D_120   395630 non-null  float64
 8   D_126   395630 non-null  float64
 9   D_66    395630 non-null  float64
 10  D_68    395630 non-null  float64
dtypes: float64(11)
memory usage: 36.2 MB


In [24]:
from sklearn.model_selection import train_test_split


In [25]:
X = df.loc[:, ~df.columns.isin(['customer_ID', 'S_2', 'target'])]
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Columns: 188 entries, P_2 to D_145
dtypes: float64(187), int64(1)
memory usage: 570.5 MB


In [26]:
Y = df.loc[:, df.columns == 'target']
Y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   target  395630 non-null  int64
dtypes: int64(1)
memory usage: 6.0 MB


In [27]:
 X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [28]:
X_train.shape

(316504, 188)

In [29]:
X_test.shape

(79126, 188)

In [30]:
from sklearn.linear_model import SGDClassifier

In [31]:
clf = SGDClassifier(verbose=1000)

In [32]:
clf.fit(X_train, y_train.values.ravel())

-- Epoch 1
Norm: 69.18, NNZs: 188, Bias: -14.485816, T: 316504, Avg. loss: 21.537350
Total training time: 0.23 seconds.
-- Epoch 2
Norm: 14.20, NNZs: 188, Bias: -8.795694, T: 633008, Avg. loss: 11.307669
Total training time: 0.44 seconds.
-- Epoch 3
Norm: 10.43, NNZs: 188, Bias: -6.661933, T: 949512, Avg. loss: 2.047519
Total training time: 0.66 seconds.
-- Epoch 4
Norm: 14.10, NNZs: 188, Bias: -4.873712, T: 1266016, Avg. loss: 3.492235
Total training time: 0.88 seconds.
-- Epoch 5
Norm: 9.89, NNZs: 188, Bias: -4.162255, T: 1582520, Avg. loss: 1.250845
Total training time: 1.10 seconds.
-- Epoch 6
Norm: 15.90, NNZs: 188, Bias: -3.839442, T: 1899024, Avg. loss: 1.040469
Total training time: 1.32 seconds.
-- Epoch 7
Norm: 7.99, NNZs: 188, Bias: -3.345688, T: 2215528, Avg. loss: 0.993473
Total training time: 1.54 seconds.
-- Epoch 8
Norm: 6.17, NNZs: 188, Bias: -2.762228, T: 2532032, Avg. loss: 0.984638
Total training time: 1.76 seconds.
-- Epoch 9
Norm: 11.12, NNZs: 188, Bias: -2.494948,

SGDClassifier(verbose=1000)

In [33]:
y_predict = clf.predict(X_test)

In [35]:
clf.score(X_test, y_test)

0.8252913075348178

In [36]:
clf.get_params()

{'alpha': 0.0001,
 'average': False,
 'class_weight': None,
 'early_stopping': False,
 'epsilon': 0.1,
 'eta0': 0.0,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'learning_rate': 'optimal',
 'loss': 'hinge',
 'max_iter': 1000,
 'n_iter_no_change': 5,
 'n_jobs': None,
 'penalty': 'l2',
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'tol': 0.001,
 'validation_fraction': 0.1,
 'verbose': 1000,
 'warm_start': False}

In [39]:
clf.decision_function(X_test)

array([ 1.44443768,  0.6856315 , -3.42618667, ...,  2.58778065,
        0.18639775, -2.87680599])

In [42]:
df_tmp = pd.read_parquet('/content/drive/MyDrive/amex-default-prediction/data/raw/data_2017-04.parquet')
df_tmp = append_label(df_tmp)
df_tmp[['D_63', 'D_64']]=enc.transform(df_tmp[['D_63', 'D_64']])
df_tmp.fillna(0, inplace=True)
X_tmp = df.loc[:, ~df_tmp.columns.isin(['customer_ID', 'S_2', 'target'])]
Y_tmp = df.loc[:, df_tmp.columns == 'target']


In [44]:
clf.score(X_tmp, Y_tmp)

0.826150696357708